In [ ]:
import numpy as np
import matplotlib.pyplot as plt
%pylab inline

from MultiArmedBandit import Arm, bernoulliArm, evolvingBernoulliArm
from Exp3 import exp3_Bianchi, exp3P_Bianchi, exp3_IX
from OtherBanditAlgorithms import UCB1, Random

In [ ]:
def gaussian_traj(T):
    """gaussian trajectories normalized to lie in [0, 1]"""
    gp = np.cumsum(np.random.normal(0, 2, T))

    # normalization (to have rewards in [0, 1]) :
    gp += np.abs(min(gp))
    gp /= max(gp)
    return(gp)

In [ ]:
T = 30000

arm1 = Arm(gaussian_traj(T))
arm2 = Arm(gaussian_traj(T))
arm3 = Arm(gaussian_traj(T))

MAB = [arm1, arm2, arm3]

plt.plot(arm1.rewards)
plt.plot(arm2.rewards)
plt.plot(arm3.rewards)
plt.legend(['Arm 1', 'Arm 2', 'Arm 3'])
plt.title('Rewards')

In [ ]:
# Compute best arm at round t
arm1_cumsum = np.cumsum(arm1.rewards)
arm2_cumsum = np.cumsum(arm2.rewards)
arm3_cumsum = np.cumsum(arm3.rewards)

best_action_rew = np.zeros(T)

for t in range(T):
    best_action_rew[t] = max(arm1_cumsum[t], arm2_cumsum[t], arm3_cumsum[t])

In [ ]:
## Weak regret bound of Exp3 if the number of rounds is known

In [ ]:
K = len(MAB)
eta = np.sqrt(2 * np.log(K) / T * K)
etas = [eta for _ in range(T)]

rew_exp3 = np.zeros(T)
n_iter = 20

for i in range(n_iter):
    rew, _ = exp3_Bianchi(MAB, T, etas)
    rew_exp3 += rew
    
rew_exp3 /= n_iter
    
exp3_cumsum = np.cumsum(rew_exp3)
plt.plot(exp3_cumsum - best_action_rew, '--o', markevery=1500, label="Exp3")

plt.axhline(np.sqrt( 2 * T * K * np.log(K)), label="weak regret bound")
plt.legend(loc="upper left")

In [ ]:
## Weak regret of Exp3 when the number of rounds is uknown

In [ ]:
K = len(MAB)
etas = [np.sqrt(2 * np.log(K) / (t + 1) * K) for t in range(T)]

rew_exp3 = np.zeros(T)
n_iter = 1000

for i in range(n_iter):
    rew_exp3 += exp3_Bianchi(MAB, T, etas)
rew_exp3 /= n_iter

    
exp3_cumsum = np.cumsum(rew_exp3)
plt.plot(exp3_cumsum - best_action_rew, '--o', markevery=1500, label="Exp3")

plt.plot([np.sqrt(4 * t * K * np.log(K)) for t in range(T)], label="weak regret bound")
plt.legend(loc="upper left")

In [ ]:
## Regret of Exp3.P

In [ ]:
K = len(MAB)
delta = 0.05
eta = 0.95 * np.sqrt(np.log(K) / (T * K))
gamma = 1.05 * np.sqrt(np.log(K) * K / T)
beta = np.sqrt(np.log(K / delta) /  (T * K))

#calculate best arm at each round
best_action_rew = np.zeros(T)
for t in range(T):
    best_action_rew[t] = max(arm1_cumsum[t], arm2_cumsum[t], arm3_cumsum[t])

#first type of regret bound
rew_exp3P = exp3P_Bianchi(MAB, T, eta, gamma, beta)

exp3P_cumsum = np.cumsum(rew_exp3P)

plt.plot(exp3P_cumsum - best_action_rew, label="Exp3P 1") 
plt.axhline(5.15 * np.sqrt(T * K * np.log(K / delta)), c="black", label="Regret 1 bound")

#second type of regret bound
beta = np.sqrt(np.log(K) /  (T * K))

rew_exp3P = exp3P_Bianchi(MAB, T, eta, gamma, beta)
exp3P_cumsum = np.cumsum(rew_exp3P)
plt.plot(exp3P_cumsum - best_action_rew, label="Exp3P 2") 
plt.axhline(5.15 * np.sqrt(T * K * np.log(K)) + np.sqrt(T * K / np.log(K)) * np.log(1 / delta), c="red", label="Regret 2 bound")

plt.legend(loc="upper left")
plt.title("Regret of Exp3.P and its bounds")

In [ ]:
## Weak regret of Exp3.P

In [ ]:
#hyperparameters
eta = 0.95 * np.sqrt(np.log(K) / (T * K))
gamma = 1.05 * np.sqrt(np.log(K) * K / T)
beta = np.sqrt(np.log(K) /  (T * K))

#calculate best arm at each round
best_action_rew = np.zeros(T)
for t in range(T):
    best_action_rew[t] = max(arm1_cumsum[t], arm2_cumsum[t], arm3_cumsum[t])

rew_exp3P = np.zeros(T)
n_iter = 100
for i in range(n_iter):
    rew_exp3P += exp3P_Bianchi(MAB, T, eta, gamma, beta)
rew_exp3P /= n_iter

exp3P_cumsum = np.cumsum(rew_exp3P)

plt.plot(exp3P_cumsum - best_action_rew, label="Exp3P 2") 
plt.axhline(5.15 * np.sqrt(T * K * np.log(K)) + np.sqrt(T * K / np.log(K)), c="red", label="Weak regret bound")

plt.legend(loc="upper left")

In [ ]:
### Compairison of different algorithms on gaussian arms

In [ ]:
#set up the parameters

K = len(MAB)
eta = np.sqrt(2 * np.log(K) / T * K)
etas_exp3 = [eta for _ in range(T)]

delta = 0.05
eta = 0.95 * np.sqrt(np.log(K) / (T * K))
gamma = 1.05 * np.sqrt(np.log(K) * K / T)
beta = np.sqrt(np.log(K / delta) /  (T * K))

In [ ]:
rew = exp3_Bianchi(MAB, T, eta=etas)
rew_P = exp3P_Bianchi(MAB, T, beta=beta, gamma=gamma, eta=eta)
rew_IX, _ = exp3_IX(MAB, T, eta=eta, gamma=0.5)
rew_UCB, _ = UCB1(MAB, T, rho=0.2)
rew_random = Random(MAB, T)

In [ ]:
arm1_cumsum = np.cumsum(arm1.rewards)
arm2_cumsum = np.cumsum(arm2.rewards)
arm3_cumsum = np.cumsum(arm3.rewards)
exp3_cumsum = np.cumsum(rew)
exp3_P_cumsum = np.cumsum(rew_P)
exp3_IX_cumsum = np.cumsum(rew_IX)
UCB_cumsum = np.cumsum(rew_UCB)
rew_cumsum = np.cumsum(rew_random)

plt.plot(arm1_cumsum, alpha=0.5, label="Arm 1")
plt.plot(arm2_cumsum, alpha=0.5, label="Arm 2")
plt.plot(arm3_cumsum, alpha=0.5, label="Arm 3")

plt.plot(exp3_cumsum, '--o', markevery=1500, label="Exp3")

plt.plot(exp3_P_cumsum, '--o', markevery=1500, label="Exp3.P")
plt.plot(exp3_IX_cumsum, '--o', markevery=1500, label="Exp3-IX")
plt.plot(UCB_cumsum, '--o', markevery=1500, label="UCB")
plt.plot(rew_cumsum, '--o', markevery=1500, label="random")

plt.legend(loc="upper left")
plt.title("Cumulative reward")